In [188]:
import numpy as np
import os
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import KFold
import numpy as np
import itertools
import time
import random

In [189]:
def load_data(folder_path):
    x_train = np.load(os.path.join(folder_path, 'x_train.npy'))
    y_train = np.load(os.path.join(folder_path, 'y_train.npy'))
    x_test = np.load(os.path.join(folder_path, 'x_test.npy'))
    y_test = np.load(os.path.join(folder_path, 'y_test.npy'))
    return x_train, y_train, x_test, y_test

In [190]:
from google.colab import drive
drive.mount('/content/drive')
x_train, y_train, x_test, y_test = load_data('/content/drive/MyDrive/lr4_dataset/')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


В данной лабораторной работе будет практиковаться поиск гиперпараметров. Буду рассмотрены алгоритмы поиска гиперпараметров: grid search, random search.

Помимо поиска гиперпараметров будет рассмотрен алгоритм кросс-валидации, позволяющий получить более достоверную оценку качества модели в условиях недостатка данных.
Хотя в работе предоставлена тестовая выборка, здесь она имеет сугубо теоретический характер (для получения финальной оценки) и на практике как правило недоступна. Поэтому во время подбора гиперпараметров используются лишь `x_train, y_train`. `x_test, y_test` используются лишь для получения финальной оценки, чтобы можно было видеть разницу между разными алгоритмами подбора гиперпараметров (если она будет).

Выберите одну модель из списка: MLPClassifier, SGDClassifier, DecisionTreeClassifier, RandomForestClassifier, SVC.
Для выбранной модели произведите поиск оптимальных гиперпараметров.

**Требование**: поиск должен идти как минимум для двух гиперпараметров.

**Требование**: в конструктор моделей передавайте `random_state=1` для воспроизводимости результатов.

## 0. Обучение бейзлайн модели для проведения сравнения

In [191]:
# Обучите базовую модель без изменения гиперпараметров (т.е. используются гиперпараметры по умолчанию).
# Проанализируйте качество модели (accuracy, матрица ошибок).

baseline_model = SVC(random_state=1)
baseline_model.fit(x_train, y_train)

y_pred = baseline_model.predict(x_test)

accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)

print(f"Точность базовой модели: {accuracy}")
print("Матрица ошибок базовой модели:")
print(conf_matrix)

Точность базовой модели: 0.7333333333333333
Матрица ошибок базовой модели:
[[3 0 0 0 0 0 0 0 0 0]
 [0 3 0 0 0 0 0 0 0 0]
 [0 0 2 1 0 0 0 0 0 0]
 [0 0 1 2 0 0 0 0 0 0]
 [0 0 0 0 3 0 0 0 0 0]
 [0 0 0 1 0 2 0 0 0 0]
 [1 0 0 0 0 0 2 0 0 0]
 [0 0 0 0 0 0 0 3 0 0]
 [0 0 1 0 0 1 0 0 1 0]
 [0 0 1 0 0 0 0 1 0 1]]


## 1. K-Fold Cross-Validation

In [192]:
# Реализуйте фунцию кросс-валидации

# Замечание: x_test, y_test не должны применятся в рамках данной функции.

def kfold_cv(model_fn, eval_fn, x: np.ndarray, y: np.ndarray, n_splits=5) -> float:
    """
    Parameters
    ----------
    model_fn : callable
        Функция-фабрика, что конструирует и возвращает новый объект модели.
        Например: `lambda: MLPClassifier(hidden_layer_sizes=(256,))`.
    eval_fn : callable
        Функция вида `eval_fn(labels, predictions)`, что возвращает скаляр (значение метрики).
    x : np.ndarray
        Набор признаков (размерность NxD, N - количество экземпляров, D - количество признаков).
    y : np.ndarray
        Набор меток (размерность N)
    n_splits : int, optional
        Количество фолдов (подвыборок), по умолчанию 5.

    Returns
    -------
    float
        Среднее значение метрики (что вычисляется eval_fn) по фолдам.
    """
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=1)
    scores = []

    for train_index, val_index in kf.split(x):
        x_train, x_val = x[train_index], x[val_index]
        y_train, y_val = y[train_index], y[val_index]
        model = model_fn()
        model.fit(x_train, y_train)
        y_pred = model.predict(x_val)
        metric_score = eval_fn(y_val, y_pred)
        #print(metric_score)
        scores.append(metric_score)
    return np.mean(scores)

In [193]:
# Убедитесь в корректности работы функции кросс-валидации.

mean_score = kfold_cv(SVC, accuracy_score, x_train, y_train)
print(f"Средняя точность по кросс-валидации: {mean_score}")

Средняя точность по кросс-валидации: 0.5


## 2. Grid search

In [194]:
# 1. Реализуйте алгоритм поиска гиперпараметров grid search.
# 2. Запустите поиск гиперпараметров, замерьте время работы алгоритма.
# 3. Выведите найденные значения гиперпараметров и время работы.
# Замечание: x_test, y_test не должны применятся в рамках данного алгоритма.
# Замечание: убедитесь, что гиперпараметры по умолчанию включены в пространство поиска.
# Требование: используйте kfold_cv для получения значения метрики в рамках одной итерации поиска гиперпараметров.

def grid_search(model_fn, param_grid, eval_fn, x: np.ndarray, y: np.ndarray, n_splits=5):
    best_score = 0
    best_params = None

    param_names = list(param_grid.keys())
    param_values = list(param_grid.values())
    param_combinations = list(itertools.product(*param_values))

    for param_set in param_combinations:
        params = dict(zip(param_names, param_set))

        def model_with_params():
            return model_fn(**params)

        score = kfold_cv(model_with_params, eval_fn, x, y, n_splits)

        if score > best_score:
            best_score = score
            best_params = params

    return best_params, best_score

param_grid = {
    'C': [0.1, 1, 5, 10],
    'kernel': ['rbf', 'poly', 'linear'],
    'gamma': ['scale', 'auto']
}

start_time = time.time()
best_params, best_score = grid_search(SVC, param_grid, accuracy_score, x_train, y_train, n_splits=5)
end_time = time.time()

print(f"Лучшие гиперпараметры: {best_params}")
print(f"Лучшая точность: {best_score}")
print(f"Время выполнения: {end_time - start_time:.2f} секунд")

Лучшие гиперпараметры: {'C': 5, 'kernel': 'rbf', 'gamma': 'scale'}
Лучшая точность: 0.5999999999999999
Время выполнения: 0.51 секунд


In [195]:
# Используйте найденные гиперпараметры для обучения модели.
# Протестируйте модель на x_test, y_test.
# Сравните полученные результаты с теми, что получены в пункте 0.   --- Результаты стали лучше

optimized_model = SVC(C=best_params['C'], kernel=best_params['kernel'], gamma=best_params['gamma'])
optimized_model.fit(x_train, y_train)

y_test_pred = optimized_model.predict(x_test)
optimized_accuracy = accuracy_score(y_test, y_test_pred)
optimized_conf_matrix = confusion_matrix(y_test, y_test_pred)

print(f"Точность модели с найденными гиперпараметрами: {optimized_accuracy}")
print("Матрица ошибок модели с найденными гиперпараметрами:")
print(optimized_conf_matrix)


print(f"\nТочность базовой модели: {accuracy}")
print("Матрица ошибок базовой модели:")
print(conf_matrix)

Точность модели с найденными гиперпараметрами: 0.8
Матрица ошибок модели с найденными гиперпараметрами:
[[3 0 0 0 0 0 0 0 0 0]
 [0 3 0 0 0 0 0 0 0 0]
 [0 0 2 1 0 0 0 0 0 0]
 [0 0 1 2 0 0 0 0 0 0]
 [0 0 0 0 3 0 0 0 0 0]
 [0 0 0 0 0 3 0 0 0 0]
 [1 0 0 0 0 1 1 0 0 0]
 [0 0 0 0 0 0 0 3 0 0]
 [0 0 0 0 0 0 0 0 3 0]
 [0 0 1 0 0 0 0 1 0 1]]

Точность базовой модели: 0.7333333333333333
Матрица ошибок базовой модели:
[[3 0 0 0 0 0 0 0 0 0]
 [0 3 0 0 0 0 0 0 0 0]
 [0 0 2 1 0 0 0 0 0 0]
 [0 0 1 2 0 0 0 0 0 0]
 [0 0 0 0 3 0 0 0 0 0]
 [0 0 0 1 0 2 0 0 0 0]
 [1 0 0 0 0 0 2 0 0 0]
 [0 0 0 0 0 0 0 3 0 0]
 [0 0 1 0 0 1 0 0 1 0]
 [0 0 1 0 0 0 0 1 0 1]]


## 3. Random search

In [196]:
# 1. Реализуйте алгоритм поиска гиперпараметров random search.
# 2. Запустите поиск гиперпараметров, замерьте время работы алгоритма.
# 3. Выведите найденные значения гиперпараметров и время работы.
# Замечание: x_test, y_test не должны применятся в рамках данного алгоритма.
# Замечание: убедитесь, что гиперпараметры по умолчанию включены в пространство поиска.
# Требование: используйте kfold_cv для получения значения метрики в рамках одной итерации поиска гиперпараметров.
# Требование: количество итераций должно быть меньше в сравнении с grid search.

def random_search(param_grid, model_fn, eval_fn, x: np.ndarray, y: np.ndarray, n_splits=5, n_iter=10):
    best_score = 0
    best_params = None

    param_names = list(param_grid.keys())
    param_values = list(param_grid.values())

    for _ in range(n_iter):
        params = {param: random.choice(values) for param, values in param_grid.items()}

        def model_with_params():
            return model_fn(**params)

        score = kfold_cv(model_with_params, eval_fn, x, y, n_splits)

        if score > best_score:
            best_score = score
            best_params = params

    return best_params, best_score

param_grid = {
    'C': [0.1, 1, 5, 10],
    'kernel': ['rbf', 'poly', 'linear'],
    'gamma': ['scale', 'auto']
}

start_time = time.time()
best_params_r, best_score_r = random_search(param_grid, SVC, accuracy_score, x_train, y_train)
end_time = time.time()

print(f"Лучшие гиперпараметры: {best_params_r}")
print(f"Лучшая точность: {best_score_r}")
print(f"Время выполнения: {end_time - start_time:.2f} секунд")

Лучшие гиперпараметры: {'C': 5, 'kernel': 'rbf', 'gamma': 'scale'}
Лучшая точность: 0.5999999999999999
Время выполнения: 0.21 секунд


In [197]:
# Используйте найденные гиперпараметры для обучения модели.
# Протестируйте модель на x_test, y_test (accuracy, матрица ошибок).
# Сравните полученные результаты с теми, что получены в пункте 0.
# Сравните полученные результаты с теми, что получены в пункте 2.

optimized_model_r = SVC(C=best_params_r['C'], kernel=best_params_r['kernel'], gamma=best_params_r['gamma'])
optimized_model_r.fit(x_train, y_train)

y_test_pred = optimized_model_r.predict(x_test)
optimized_accuracy_r = accuracy_score(y_test, y_test_pred)
optimized_conf_matrix_r = confusion_matrix(y_test, y_test_pred)

print("Random search:")
print(f"Точность модели с найденными гиперпараметрами: {optimized_accuracy_r}")
print("Матрица ошибок модели с найденными гиперпараметрами:")
print(optimized_conf_matrix_r)


print(f"\nТочность базовой модели: {accuracy}")
print("Матрица ошибок базовой модели:")
print(conf_matrix)

print("\nGrid search:")
print(f"Точность модели с найденными гиперпараметрами: {optimized_accuracy}")
print("Матрица ошибок модели с найденными гиперпараметрами:")
print(optimized_conf_matrix)

Random search:
Точность модели с найденными гиперпараметрами: 0.8
Матрица ошибок модели с найденными гиперпараметрами:
[[3 0 0 0 0 0 0 0 0 0]
 [0 3 0 0 0 0 0 0 0 0]
 [0 0 2 1 0 0 0 0 0 0]
 [0 0 1 2 0 0 0 0 0 0]
 [0 0 0 0 3 0 0 0 0 0]
 [0 0 0 0 0 3 0 0 0 0]
 [1 0 0 0 0 1 1 0 0 0]
 [0 0 0 0 0 0 0 3 0 0]
 [0 0 0 0 0 0 0 0 3 0]
 [0 0 1 0 0 0 0 1 0 1]]

Точность базовой модели: 0.7333333333333333
Матрица ошибок базовой модели:
[[3 0 0 0 0 0 0 0 0 0]
 [0 3 0 0 0 0 0 0 0 0]
 [0 0 2 1 0 0 0 0 0 0]
 [0 0 1 2 0 0 0 0 0 0]
 [0 0 0 0 3 0 0 0 0 0]
 [0 0 0 1 0 2 0 0 0 0]
 [1 0 0 0 0 0 2 0 0 0]
 [0 0 0 0 0 0 0 3 0 0]
 [0 0 1 0 0 1 0 0 1 0]
 [0 0 1 0 0 0 0 1 0 1]]

Grid search:
Точность модели с найденными гиперпараметрами: 0.8
Матрица ошибок модели с найденными гиперпараметрами:
[[3 0 0 0 0 0 0 0 0 0]
 [0 3 0 0 0 0 0 0 0 0]
 [0 0 2 1 0 0 0 0 0 0]
 [0 0 1 2 0 0 0 0 0 0]
 [0 0 0 0 3 0 0 0 0 0]
 [0 0 0 0 0 3 0 0 0 0]
 [1 0 0 0 0 1 1 0 0 0]
 [0 0 0 0 0 0 0 3 0 0]
 [0 0 0 0 0 0 0 0 3 0]
 [0 0 1 0 0 0 0 1 

## 4. Доп. задание (опционально)

### 4.1 Bayesian optimization

Примените байесовскую оптимизацию для поиска гиперпараметров.
В качестве алгоритма используйте `BayesSearchCV` из пакета `scikit-optimize`.

Сложность: почти бесплатный балл.

In [198]:
!pip install scikit-optimize

In [199]:
# 1. Инстанцируйте BayesSearchCV.
# 2. Запустите поиск гиперпараметров, замерьте время работы алгоритма.
# 3. Выведите найденные значения гиперпараметров и время работы.

from skopt import BayesSearchCV

param_grid = {
    'C': [0.1, 1, 5, 10],
    'kernel': ['rbf', 'poly', 'linear'],
    'gamma': ['scale', 'auto']
}

bayes_search = BayesSearchCV(SVC(), param_grid, n_iter=10, cv=5)

start_time = time.time()
bayes_search.fit(x_train, y_train)
end_time = time.time()

print(f"Лучшие гиперпараметры: {bayes_search.best_params_}")
print(f"Лучшая точность: {bayes_search.best_score_:.4f}")
print(f"Время выполнения: {end_time - start_time:.2f} секунд")

Лучшие гиперпараметры: OrderedDict([('C', 5), ('gamma', 'scale'), ('kernel', 'rbf')])
Лучшая точность: 0.6273
Время выполнения: 1.11 секунд


In [200]:
# Используйте найденные гиперпараметры для обучения модели.
# Протестируйте модель на x_test, y_test (accuracy, матрица ошибок).
# Сравните полученные результаты с теми, что получены в пункте 0.
# Сравните полученные результаты с теми, что получены в пункте 2.

optimized_model_b = SVC(C=bayes_search.best_params_['C'], kernel=bayes_search.best_params_['kernel'], gamma=bayes_search.best_params_['gamma'])
optimized_model_b.fit(x_train, y_train)

y_test_pred = optimized_model_b.predict(x_test)
optimized_accuracy_b = accuracy_score(y_test, y_test_pred)
optimized_conf_matrix_b = confusion_matrix(y_test, y_test_pred)

print("Bayesian optimization:")
print(f"Точность модели с найденными гиперпараметрами: {optimized_accuracy_b}")
print("Матрица ошибок модели с найденными гиперпараметрами:")
print(optimized_conf_matrix_b)

print("\nRandom search:")
print(f"Точность модели с найденными гиперпараметрами: {optimized_accuracy_r}")
print("Матрица ошибок модели с найденными гиперпараметрами:")
print(optimized_conf_matrix_r)

print(f"\nТочность базовой модели: {accuracy}")
print("Матрица ошибок базовой модели:")
print(conf_matrix)

print("\nGrid search:")
print(f"Точность модели с найденными гиперпараметрами: {optimized_accuracy}")
print("Матрица ошибок модели с найденными гиперпараметрами:")
print(optimized_conf_matrix)

Bayesian optimization:
Точность модели с найденными гиперпараметрами: 0.8
Матрица ошибок модели с найденными гиперпараметрами:
[[3 0 0 0 0 0 0 0 0 0]
 [0 3 0 0 0 0 0 0 0 0]
 [0 0 2 1 0 0 0 0 0 0]
 [0 0 1 2 0 0 0 0 0 0]
 [0 0 0 0 3 0 0 0 0 0]
 [0 0 0 0 0 3 0 0 0 0]
 [1 0 0 0 0 1 1 0 0 0]
 [0 0 0 0 0 0 0 3 0 0]
 [0 0 0 0 0 0 0 0 3 0]
 [0 0 1 0 0 0 0 1 0 1]]

Random search:
Точность модели с найденными гиперпараметрами: 0.8
Матрица ошибок модели с найденными гиперпараметрами:
[[3 0 0 0 0 0 0 0 0 0]
 [0 3 0 0 0 0 0 0 0 0]
 [0 0 2 1 0 0 0 0 0 0]
 [0 0 1 2 0 0 0 0 0 0]
 [0 0 0 0 3 0 0 0 0 0]
 [0 0 0 0 0 3 0 0 0 0]
 [1 0 0 0 0 1 1 0 0 0]
 [0 0 0 0 0 0 0 3 0 0]
 [0 0 0 0 0 0 0 0 3 0]
 [0 0 1 0 0 0 0 1 0 1]]

Точность базовой модели: 0.7333333333333333
Матрица ошибок базовой модели:
[[3 0 0 0 0 0 0 0 0 0]
 [0 3 0 0 0 0 0 0 0 0]
 [0 0 2 1 0 0 0 0 0 0]
 [0 0 1 2 0 0 0 0 0 0]
 [0 0 0 0 3 0 0 0 0 0]
 [0 0 0 1 0 2 0 0 0 0]
 [1 0 0 0 0 0 2 0 0 0]
 [0 0 0 0 0 0 0 3 0 0]
 [0 0 1 0 0 1 0 0 1 0]
 [0 0 1 

### 4.2 Tree of Parzen Estimators (TPE) из HyperOpt

Примените TPE из библиотеки hyperopt для поиска гиперпараметров. Вики по HyperOpt: https://github.com/hyperopt/hyperopt/wiki/FMin

Сложность: чтец документаций o(*￣▽￣*)ブ.

In [201]:
!pip install hyperopt

In [208]:
from hyperopt import hp, tpe, fmin

def objective(args):
    # Принимает гиперпараметры, инстанцирует модель, обучает её, возвращает значение метрики.
    # Замечание: x_test, y_test не должны применятся в рамках данного алгоритма.
    model = SVC(C=args['C'], kernel=args['kernel'], gamma=args['gamma'])
    model.fit(x_train, y_train)
    y_pred = model.predict(x_test)
    accuracy = accuracy_score(y_test, y_pred)
    return -accuracy

In [209]:
# Определите пространство поиска гиперпараметров
space = {
    'C': hp.choice('C', [0.1, 1, 5, 10]),
    'kernel': hp.choice('kernel', ['linear', 'rbf', 'poly']),
    'gamma': hp.choice('gamma', ['scale', 'auto'])
}

In [210]:
# 1. Запустите поиск гиперпараметров, замерьте время работы алгоритма.
# 2. Выведите найденные значения гиперпараметров и время работы.

start_time = time.time()

tpe_search = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=50)

end_time = time.time()

c_mapping = [0.1, 1, 5, 10]
kernel_mapping = ['linear', 'rbf', 'poly']
gamma_mapping = ['scale', 'auto']

tpe_c = c_mapping[tpe_search['C']]
tpe_kernel = kernel_mapping[tpe_search['kernel']]
tpe_gamma = gamma_mapping[tpe_search['gamma']]

print("Лучшие гиперпараметры:", f"C: {tpe_c}, kernel: {tpe_kernel}, gamma: {tpe_gamma}")
print(f"Время выполнения: {end_time - start_time:.2f} секунд")

100%|██████████| 50/50 [00:00<00:00, 113.12trial/s, best loss: -0.8]
Лучшие гиперпараметры: C: 10, kernel: rbf, gamma: scale
Время выполнения: 0.46 секунд


In [211]:
# Используйте найденные гиперпараметры для обучения модели.
# Протестируйте модель на x_test, y_test (accuracy, матрица ошибок).
# Сравните полученные результаты с теми, что получены в пункте 0.
# Сравните полученные результаты с теми, что получены в пункте 2.

tpe_model = SVC(C=tpe_c, kernel=tpe_kernel, gamma=tpe_gamma)
tpe_model.fit(x_train, y_train)

y_pred = tpe_model.predict(x_test)

print("TPE:")
print("Точность модели с найденными гиперпараметрами:", accuracy_score(y_test, y_pred))
print("Матрица ошибок модели с найденными гиперпараметрами:")
print(confusion_matrix(y_test, y_pred))

print("\nBayesian optimization:")
print(f"Точность модели с найденными гиперпараметрами: {optimized_accuracy_b}")
print("Матрица ошибок модели с найденными гиперпараметрами:")
print(optimized_conf_matrix_b)

print("\nRandom search:")
print(f"Точность модели с найденными гиперпараметрами: {optimized_accuracy_r}")
print("Матрица ошибок модели с найденными гиперпараметрами:")
print(optimized_conf_matrix_r)

print(f"\nТочность базовой модели: {accuracy}")
print("Матрица ошибок базовой модели:")
print(conf_matrix)

print("\nGrid search:")
print(f"Точность модели с найденными гиперпараметрами: {optimized_accuracy}")
print("Матрица ошибок модели с найденными гиперпараметрами:")
print(optimized_conf_matrix)

TPE:
Точность модели с найденными гиперпараметрами: 0.8
Матрица ошибок модели с найденными гиперпараметрами:
[[3 0 0 0 0 0 0 0 0 0]
 [0 3 0 0 0 0 0 0 0 0]
 [0 0 2 1 0 0 0 0 0 0]
 [0 0 1 2 0 0 0 0 0 0]
 [0 0 0 0 3 0 0 0 0 0]
 [0 0 0 0 0 3 0 0 0 0]
 [1 0 0 0 0 1 1 0 0 0]
 [0 0 0 0 0 0 0 3 0 0]
 [0 0 0 0 0 0 0 0 3 0]
 [0 0 1 0 0 0 0 1 0 1]]

Bayesian optimization:
Точность модели с найденными гиперпараметрами: 0.8
Матрица ошибок модели с найденными гиперпараметрами:
[[3 0 0 0 0 0 0 0 0 0]
 [0 3 0 0 0 0 0 0 0 0]
 [0 0 2 1 0 0 0 0 0 0]
 [0 0 1 2 0 0 0 0 0 0]
 [0 0 0 0 3 0 0 0 0 0]
 [0 0 0 0 0 3 0 0 0 0]
 [1 0 0 0 0 1 1 0 0 0]
 [0 0 0 0 0 0 0 3 0 0]
 [0 0 0 0 0 0 0 0 3 0]
 [0 0 1 0 0 0 0 1 0 1]]

Random search:
Точность модели с найденными гиперпараметрами: 0.8
Матрица ошибок модели с найденными гиперпараметрами:
[[3 0 0 0 0 0 0 0 0 0]
 [0 3 0 0 0 0 0 0 0 0]
 [0 0 2 1 0 0 0 0 0 0]
 [0 0 1 2 0 0 0 0 0 0]
 [0 0 0 0 3 0 0 0 0 0]
 [0 0 0 0 0 3 0 0 0 0]
 [1 0 0 0 0 1 1 0 0 0]
 [0 0 0 0 0 0 0 3 0 